<h1> Template Matching - Normalized Cross Correlation - Dual Check </h1>

In [ ]:
import json, os
import cv2
from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils
from aquabyte.visualize import Visualizer
from aquabyte.template_matching import enhance
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import Image
from PIL import Image, ImageDraw
from collections import defaultdict
from aquabyte.optics import euclidean_distance, pixel2world, depth_from_disp, convert_to_world_point
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

In [ ]:
# extract sample dataset
s3_access_utils = S3AccessUtils('/root/data')
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))

query = """
    SELECT * FROM keypoint_annotations
    WHERE pen_id = 56
    AND keypoints -> 'leftCrop' is not null
    AND keypoints -> 'rightCrop' is not null
    limit 1000;
"""

df = rds_access_utils.extract_from_database(query)



<h1> Visualize some keypoint annotations </h1>

In [ ]:
v = Visualizer(s3_access_utils, rds_access_utils)

In [ ]:
keypoint_annotation_id = int(df.id.iloc[0])
v.load_data(keypoint_annotation_id)
v.display_crops()

<h1> Attempt ADIPOSE_FIN correction via normalized cross correlation </h1>

In [ ]:
body_part = 'EYE'
half_box_size = 25
horizontal_search_range = 30
vertical_search_range = 5

# download left and right crops
mask = df.id == keypoint_annotation_id
left_crop_url = df[mask].left_image_url.iloc[0]
left_crop_f, _, _ = s3_access_utils.download_from_url(left_crop_url)
left_crop_image = Image.open(left_crop_f)
left_crop_arr = enhance(np.array(left_crop_image))

right_crop_url = df[mask].right_image_url.iloc[0]
right_crop_f, _, _ = s3_access_utils.download_from_url(right_crop_url)
right_crop_image = Image.open(right_crop_f)
right_crop_arr = enhance(np.array(right_crop_image))

# get keypoint coordinates
keypoints = df[mask].keypoints.iloc[0]
left_kps = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in keypoints['leftCrop']}
right_kps = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in keypoints['rightCrop']}

# jitter keypoints
# left_kps = {bp: left_kps[bp] + np.random.normal(0, 10, 2).astype(int) for bp in left_kps.keys()}
# right_kps = {bp: right_kps[bp] + np.random.normal(0, 10, 2).astype(int) for bp in right_kps.keys()}

# direction 1
template = left_crop_arr[left_kps[body_part][1]-half_box_size:left_kps[body_part][1]+half_box_size, 
                         left_kps[body_part][0]-half_box_size:left_kps[body_part][0]+half_box_size]
source = right_crop_arr[right_kps[body_part][1]-half_box_size-vertical_search_range:right_kps[body_part][1]+half_box_size+vertical_search_range, 
                        right_kps[body_part][0]-half_box_size-horizontal_search_range:right_kps[body_part][0]+half_box_size+horizontal_search_range]
print(template.sum(), source.sum())

template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
source_gray = cv2.cvtColor(source, cv2.COLOR_BGR2GRAY)
res = cv2.matchTemplate(source_gray, template_gray, cv2.TM_CCOEFF_NORMED)
a, b = np.unravel_index(np.argmax(res, axis=None), res.shape)
adj_right_keypoint = np.array([right_kps[body_part][0] - horizontal_search_range + b, 
                      right_kps[body_part][1] - vertical_search_range + a])

adj_right = adj_right_keypoint - right_kps[body_part]

# direction 2
template = right_crop_arr[right_kps[body_part][1]-half_box_size:right_kps[body_part][1]+half_box_size, 
                          right_kps[body_part][0]-half_box_size:right_kps[body_part][0]+half_box_size]

source = left_crop_arr[left_kps[body_part][1]-half_box_size-vertical_search_range:left_kps[body_part][1]+half_box_size+vertical_search_range, 
                       left_kps[body_part][0]-half_box_size-horizontal_search_range:left_kps[body_part][0]+half_box_size+horizontal_search_range]

print(template.sum(), source.sum())
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
source_gray = cv2.cvtColor(source, cv2.COLOR_BGR2GRAY)
res = cv2.matchTemplate(source_gray, template_gray, cv2.TM_CCOEFF_NORMED)
a, b = np.unravel_index(np.argmax(res, axis=None), res.shape)
adj_left_keypoint = np.array([left_kps[body_part][0] - horizontal_search_range + b, 
                      left_kps[body_part][1] - vertical_search_range + a])

adj_left = adj_left_keypoint - left_kps[body_part]

In [ ]:
print(np.linalg.norm(adj_right + adj_left))
print(0.5 * (np.linalg.norm(adj_left) + np.linalg.norm(adj_right)))
print(np.dot(adj_left, adj_right)/(np.linalg.norm(adj_left) * np.linalg.norm(adj_right)))

In [ ]:
template.sum()

In [ ]:
left_kps[body_part][1]-half_box_size-vertical_search_range,left_kps[body_part][1]+half_box_size+vertical_search_range, left_kps[body_part][0]-half_box_size-horizontal_search_range,left_kps[body_part][0]+half_box_size+horizontal_search_range
        
        

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
right_image = plt.imread(right_crop_f)
ax.imshow(right_image)
ax.scatter([right_kps[body_part][0]], [right_kps[body_part][1]], color='red', s=1)
ax.scatter([adj_right_keypoint[0]], [adj_right_keypoint[1]], color='green', s=1)
plt.show()

In [ ]:
body_parts = ['UPPER_LIP',
 'EYE',
 'DORSAL_FIN',
 'ADIPOSE_FIN',
 'UPPER_PRECAUDAL_PIT',
 'HYPURAL_PLATE',
 'TAIL_NOTCH',
 'LOWER_PRECAUDAL_PIT',
 'ANAL_FIN',
 'PELVIC_FIN',
 'PECTORAL_FIN'
]


half_box_size = 35
horizontal_search_range = 50
vertical_search_range = 5

In [ ]:
def get_patch_arr(im_arr, kp, half_box_size, horizontal_search_range=0, vertical_search_range=0):
    patch_coord_1_min = kp[1]-half_box_size-vertical_search_range
    patch_coord_1_max = kp[1]+half_box_size+vertical_search_range
    if patch_coord_1_min < 0:
        patch_coord_1_min, patch_coord_1_max = 0, 2*(half_box_size+vertical_search_range)
    elif patch_coord_1_max > im_arr.shape[0]-1:
        patch_coord_1_min, patch_coord_1_max = im_arr.shape[0]-1-2*(half_box_size+vertical_search_range), \
                                                  im_arr.shape[0]-1

    patch_coord_2_min = kp[0]-half_box_size-horizontal_search_range
    patch_coord_2_max = kp[0]+half_box_size+horizontal_search_range
    if patch_coord_2_min < 0:
        patch_coord_2_min, patch_coord_2_max = 0, 2*(half_box_size+horizontal_search_range)
    elif patch_coord_2_max > im_arr.shape[1]-1:
        patch_coord_2_min, patch_coord_2_max = im_arr.shape[1]-1-2*(half_box_size+horizontal_search_range), \
                                               im_arr.shape[1]-1

    patch = im_arr[patch_coord_1_min:patch_coord_1_max, patch_coord_2_min:patch_coord_2_max]
    return patch



In [ ]:
analysis_data, keypoint_data = defaultdict(list), defaultdict(list)
count = 0
for idx, row in df.head(1000).iterrows():
    
    # download left and right crops
    left_crop_url = row.left_image_url
    left_crop_f, _, _ = s3_access_utils.download_from_url(left_crop_url)
    left_crop_image = Image.open(left_crop_f)
    left_crop_arr = enhance(np.array(left_crop_image))

    right_crop_url = row.right_image_url
    right_crop_f, _, _ = s3_access_utils.download_from_url(right_crop_url)
    right_crop_image = Image.open(right_crop_f)
    right_crop_arr = enhance(np.array(right_crop_image))

    # get keypoint coordinates
    keypoints = row.keypoints
    original_left_kps = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in keypoints['leftCrop']}
    original_right_kps = {item['keypointType']: np.array([item['xCrop'], item['yCrop']]) for item in keypoints['rightCrop']}
    
    # jitter keypoints
    left_kps = {bp: original_left_kps[bp] + np.array([int(np.random.normal(0, 15)), 0]) for bp in original_left_kps.keys()}
    right_kps = {bp: original_right_kps[bp] + np.array([int(np.random.normal(0, 15)), 0]) for bp in original_right_kps.keys()}
    
    left_kps_frame = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in keypoints['leftCrop']}
    right_kps_frame = {item['keypointType']: np.array([item['xFrame'], item['yFrame']]) for item in keypoints['rightCrop']}
    
    adj_left_kps, adj_right_kps = {}, {}
    
    for body_part in body_parts:
        
        # direction 1
        template = get_patch_arr(left_crop_arr, left_kps[body_part], half_box_size)
        
        source = get_patch_arr(right_crop_arr, right_kps[body_part], half_box_size, \
                                horizontal_search_range=horizontal_search_range, \
                                vertical_search_range=vertical_search_range)

        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
        source_gray = cv2.cvtColor(source, cv2.COLOR_BGR2GRAY)
        res = cv2.matchTemplate(source_gray, template_gray, cv2.TM_CCOEFF_NORMED)
        a, b = np.unravel_index(np.argmax(res, axis=None), res.shape)
        adj_right_keypoint = np.array([right_kps[body_part][0] - horizontal_search_range + b, 
                              right_kps[body_part][1] - vertical_search_range + a])
        
        template_2 = get_patch_arr(right_crop_arr, adj_right_keypoint, half_box_size)
        source_2 = get_patch_arr(left_crop_arr, left_kps[body_part], half_box_size, \
                                horizontal_search_range=horizontal_search_range, \
                                vertical_search_range=vertical_search_range)
        
        template_2_gray = cv2.cvtColor(template_2, cv2.COLOR_BGR2GRAY)
        source_2_gray = cv2.cvtColor(source_2, cv2.COLOR_BGR2GRAY)
        res_2 = cv2.matchTemplate(source_2_gray, template_2_gray, cv2.TM_CCOEFF_NORMED)
        a_2, b_2 = np.unravel_index(np.argmax(res_2, axis=None), res_2.shape)
        backproj_left_keypoint = np.array([left_kps[body_part][0] - horizontal_search_range + b_2, 
                              left_kps[body_part][1] - vertical_search_range + a_2])

        net_magnitude_1 = np.linalg.norm(left_kps[body_part] - backproj_left_keypoint)
        print(net_magnitude_1, res.max(), res_2.max())

        # direction 2
        template = get_patch_arr(right_crop_arr, right_kps[body_part], half_box_size)
        source = get_patch_arr(left_crop_arr, left_kps[body_part], half_box_size, \
                                horizontal_search_range=horizontal_search_range, \
                                vertical_search_range=vertical_search_range)

        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
        source_gray = cv2.cvtColor(source, cv2.COLOR_BGR2GRAY)
        res = cv2.matchTemplate(source_gray, template_gray, cv2.TM_CCOEFF_NORMED)
        a, b = np.unravel_index(np.argmax(res, axis=None), res.shape)
        adj_left_keypoint = np.array([left_kps[body_part][0] - horizontal_search_range + b, 
                              left_kps[body_part][1] - vertical_search_range + a])
        
        template_2 = get_patch_arr(left_crop_arr, adj_left_keypoint, half_box_size)
        source_2 = get_patch_arr(right_crop_arr, right_kps[body_part], half_box_size, \
                                horizontal_search_range=horizontal_search_range, \
                                vertical_search_range=vertical_search_range)
        
        template_2_gray = cv2.cvtColor(template_2, cv2.COLOR_BGR2GRAY)
        source_2_gray = cv2.cvtColor(source_2, cv2.COLOR_BGR2GRAY)
        res_2 = cv2.matchTemplate(source_2_gray, template_2_gray, cv2.TM_CCOEFF_NORMED)
        a_2, b_2 = np.unravel_index(np.argmax(res_2, axis=None), res_2.shape)
        backproj_right_keypoint = np.array([right_kps[body_part][0] - horizontal_search_range + b_2, 
                                            right_kps[body_part][1] - vertical_search_range + a_2])
        
        net_magnitude_2 = np.linalg.norm(right_kps[body_part] - backproj_right_keypoint)
        
        if (net_magnitude_1 <= net_magnitude_2) and (net_magnitude_1 < 3):
            adj_left_kps[body_part] = backproj_left_keypoint
            adj_right_kps[body_part] = adj_right_keypoint
        elif (net_magnitude_2 <= net_magnitude_1) and (net_magnitude_2 < 3):
            adj_left_kps[body_part] = adj_left_keypoint
            adj_right_kps[body_part] = backproj_right_keypoint

        analysis_data['net_magnitude_1'].append(net_magnitude_1)
        analysis_data['net_magnitude_2'].append(net_magnitude_2)
        analysis_data['body_part'].append(body_part)
        analysis_data['keypoint_annotation_id'].append(row.id)
        
    keypoint_data['original_left_keypoints'].append(original_left_kps)
    keypoint_data['original_right_keypoints'].append(original_right_kps)
    keypoint_data['left_keypoints'].append(left_kps)
    keypoint_data['right_keypoints'].append(right_kps)
#     keypoint_data['backproj_left_keypoints'].append(backproj_left_kps)
#     keypoint_data['backproj_right_keypoints'].append(backproj_right_kps)
    keypoint_data['adj_left_keypoints'].append(adj_left_kps)
    keypoint_data['adj_right_keypoints'].append(adj_right_kps)
    keypoint_data['left_image_f'].append(left_crop_f)
    keypoint_data['right_image_f'].append(right_crop_f)
    keypoint_data['keypoint_annotation_id'].append(keypoint_annotation_id)

    if count % 10 == 0:
        print(count)
    count += 1
    
analysis_df = pd.DataFrame(analysis_data)


In [ ]:
analysis_df = pd.DataFrame(analysis_data)
keypoint_df = pd.DataFrame(keypoint_data)

In [ ]:
mask = analysis_df.body_part == 'PECTORAL_FIN'
well_behaved_mask = mask & ((analysis_df.net_magnitude_1 < 8) | (analysis_df.net_magnitude_2 < 8))
analysis_df[well_behaved_mask].shape[0] / analysis_df[mask].shape[0]

In [ ]:
good_adjustment_mask = (analysis_df.net_magnitude_1 < 8) | (analysis_df.net_magnitude_2 < 8)
minimal_set_present_count = 0
for keypoint_annotation_id in analysis_df.keypoint_annotation_id.unique():
    id_mask = analysis_df.keypoint_annotation_id == keypoint_annotation_id
    body_parts = analysis_df[id_mask & good_adjustment_mask].body_part.tolist()
    tail_present = any([bp in body_parts for bp in ['UPPER_PRECAUDAL_PIT', 'LOWER_PRECAUDAL_PIT', 'HYPURAL_PLATE']])
    back_present = any([bp in body_parts for bp in ['ADIPOSE_FIN', 'ANAL_FIN']])
    middle_present = all([bp in body_parts for bp in ['DORSAL_FIN', 'PELVIC_FIN']])
    front_present = all([bp in body_parts for bp in ['PECTORAL_FIN', 'EYE']])
    minimal_set_present = tail_present and back_present and middle_present and front_present
    if minimal_set_present:
        minimal_set_present_count += 1
        
print(minimal_set_present_count / len(analysis_df.keypoint_annotation_id.unique().tolist()))

In [ ]:
keypoint_df = pd.DataFrame(keypoint_data)

In [ ]:
# def display_crops(left_keypoints, right_keypoints, adj_left_keypoints, adj_right_keypoints, left_image_f, right_image_f):
def display_crops(adj_left_keypoints, adj_right_keypoints, left_image_f, right_image_f):
    fig, axes = plt.subplots(2, 1, figsize=(20, 20))
    left_image = plt.imread(left_image_f)
    right_image = plt.imread(right_image_f)
    axes[0].imshow(left_image)
    axes[1].imshow(right_image)
#     for bp, kp in left_keypoints.items():
#         axes[0].scatter([kp[0]], [kp[1]], color='red', s=1)
#     for bp, kp in right_keypoints.items():
#         axes[1].scatter([kp[0]], [kp[1]], color='red', s=1)
    for bp, kp in adj_left_keypoints.items():
        axes[0].scatter([kp[0]], [kp[1]], color='red', s=1)
    for bp, kp in adj_right_keypoints.items():
        axes[1].scatter([kp[0]], [kp[1]], color='red', s=1)
    plt.show()

In [ ]:
idx = 109
left_keypoints = keypoint_df.left_keypoints.iloc[idx]
right_keypoints = keypoint_df.right_keypoints.iloc[idx]
adj_left_keypoints = keypoint_df.adj_left_keypoints.iloc[idx]
adj_right_keypoints = keypoint_df.adj_right_keypoints.iloc[idx]
left_image_f = keypoint_df.left_image_f.iloc[idx]
right_image_f = keypoint_df.right_image_f.iloc[idx]
display_crops(adj_left_keypoints, adj_right_keypoints, left_image_f, right_image_f)
display_crops(left_keypoints, right_keypoints, left_image_f, right_image_f)